In [12]:
#import the library used to query a website
import urllib3
#import the Beautiful soup functions to parse the data returned from the website
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
'Accept-Encoding': 'none',
'Accept-Language': 'en-US,en;q=0.8',
'Connection': 'keep-alive'}



In [18]:
#first step to generate links
alllinks=[]
alllinksJOBS=""
finallinks = []

numberofpages = 20 #THIS IS IMPORTANT HOW MANY PAGES TO DOWNLOAD
for x in range(numberofpages):
    alllinks.append("https://www.naukrigulf.com/healthcare-jobs-in-uae-%d" %x)
#alllinks.append("https://www.naukrigulf.com/oil-and-gas-jobs-%d?industry=36" % x)

#iterate over all links
for links in alllinks:
    req = urllib.request.Request(links, headers=hdr)
    page = urllib.request.urlopen(req)
    naukrigulfHtml = page.read()
    page.close()

In [20]:
soup = BeautifulSoup(naukrigulfHtml, "html.parser")  # Parse the html in the 'page' variable, and store it in Beautiful Soup format
NaukriAllLinks = soup.find_all("a")  # we find only the links which start by "a" HTML code

for links in NaukriAllLinks:
    tmplink = links.get('href')  # inside all links with "a" we want only those with "href" HTML code
    alllinksJOBS = alllinksJOBS + "\n" + str(tmplink)  # make one big string will all links

text_file = open("alllinksNAUKRI.txt", "w")  # store it in txt file for further processing
text_file.write(alllinksJOBS)
text_file.close()

In [53]:
openfile = open("alllinksNAUKRI.txt")
for line in openfile:
# if re.search(r"^/pagead/clk.*", line, re.MULTILINE): #using regular expression we take only the valid links for jobs
#     finallinks.append(line.rstrip()) #store them in list for further processing (to remove duplicate links)
    if re.search(r"^https://www.naukrigulf.com/.*\d{12}$", line, flags=re.MULTILINE):  # using regular expression we take only the valid links for jobs
        finallinks.append(line.rstrip()) #store them in list for further processing (to remove duplicate links)
        #openfile.close()

#here are the JOB links from all INDEED pages
FinalJobLinks = np.unique(finallinks) #we take only unique job links

print(FinalJobLinks.size, "Jobs to download:")



25 Jobs to download:


In [89]:
df = pd.DataFrame(columns=['Title','Description']) #create empty data fr
for x in range(FinalJobLinks.size):
        print("We're on job number %d" %(x))
        NAUKRIHealthCare = FinalJobLinks[x] #specify the job url
        req2 = urllib.request.Request(NAUKRIHealthCare, headers=hdr)
        page = urllib.request.urlopen(req2) #Query the website and return the html to the variable 'page'
        soup = BeautifulSoup(page, "html.parser") #Parse the html in the 'page' variable, and store it in Beautiful Soup format
        #print(NAUKRIHealthCare)
        title = soup.h1.text  # get title


        tmplist = re.split(r"Job Type", soup.select_one("p[class*=cl]").text)
        description = tmplist[0].replace("Job Summary", '').replace("Job Description", '') #get description
        title = title.strip() #remove blank characters
        title = title.encode('utf-8') #put utf8 encoding because of arabic and some other special characters
        description = description.strip()
        description = description.encode('utf-8')
        df = df.append({
             "Title": title,
            "Description":  description}, ignore_index=True)
    

        


    
    

We're on job number 0
We're on job number 1
We're on job number 2
We're on job number 3
We're on job number 4
We're on job number 5
We're on job number 6
We're on job number 7
We're on job number 8
We're on job number 9
We're on job number 10
We're on job number 11
We're on job number 12
We're on job number 13
We're on job number 14
We're on job number 15
We're on job number 16
We're on job number 17
We're on job number 18
We're on job number 19
We're on job number 20
We're on job number 21
We're on job number 22
We're on job number 23
We're on job number 24


In [90]:

    
#print(df.loc[x])




df.to_csv('NaukriGulfhealthcare.csv')